In [71]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [72]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [73]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308937  [    0/60000]
loss: 2.299855  [ 6400/60000]
loss: 2.280784  [12800/60000]
loss: 2.273147  [19200/60000]
loss: 2.266777  [25600/60000]
loss: 2.228288  [32000/60000]
loss: 2.241891  [38400/60000]
loss: 2.211389  [44800/60000]
loss: 2.200930  [51200/60000]
loss: 2.164481  [57600/60000]
Test Error: 
 Accuracy: 32.0%, Avg loss: 2.166163 

Epoch 2
-------------------------------
loss: 2.177612  [    0/60000]
loss: 2.172463  [ 6400/60000]
loss: 2.116822  [12800/60000]
loss: 2.131139  [19200/60000]
loss: 2.094444  [25600/60000]
loss: 2.028850  [32000/60000]
loss: 2.064559  [38400/60000]
loss: 1.989377  [44800/60000]
loss: 1.981007  [51200/60000]
loss: 1.916044  [57600/60000]
Test Error: 
 Accuracy: 56.8%, Avg loss: 1.914682 

Epoch 3
-------------------------------
loss: 1.942742  [    0/60000]
loss: 1.920837  [ 6400/60000]
loss: 1.804549  [12800/60000]
loss: 1.846933  [19200/60000]
loss: 1.744292  [25600/60000]
loss: 1.692108  [32000/600

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-0.0173,  0.0169, -0.0286,  ..., -0.0314, -0.0046, -0.0226],
                      [-0.0259,  0.0158, -0.0344,  ..., -0.0040, -0.0031,  0.0091],
                      [ 0.0084, -0.0067, -0.0314,  ..., -0.0091, -0.0311,  0.0072],
                      ...,
                      [-0.0132,  0.0079,  0.0063,  ...,  0.0341, -0.0315,  0.0323],
                      [-0.0018, -0.0228,  0.0057,  ...,  0.0158, -0.0108, -0.0302],
                      [-0.0005, -0.0255, -0.0326,  ...,  0.0038, -0.0043,  0.0079]])),
             ('linear_relu_stack.0.bias',
              tensor([ 3.1702e-02, -2.1351e-02,  1.2803e-02,  3.1583e-02, -2.5227e-03,
                       2.9534e-02,  1.4413e-02,  1.6015e-04, -3.3959e-02,  1.4816e-02,
                      -2.6293e-03,  1.7806e-02,  4.2919e-02,  1.7080e-02, -2.1047e-02,
                      -1.4606e-02, -2.6954e-03, -4.0377e-03,  3.1872e-02, -2.9564e-02,
                       1.3277e-0

In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [14]:
test_data[0][0].shape

torch.Size([1, 28, 28])

In [70]:
test_data[0][1]

9

In [74]:
class_probs = []
class_label = []
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        output = model(images)
        class_probs_batch = [nn.functional.softmax(el, dim=0) for el in output]

        class_probs.append(class_probs_batch)
        class_label.append(labels)

test_probs = torch.cat([torch.stack(batch) for batch in class_probs])
test_label = torch.cat(class_label)

In [94]:
test_probs[:, 0]

tensor([0.0037, 0.0244, 0.0454,  ..., 0.2419, 0.0690, 0.0141])

In [79]:
test_probs[0]

tensor([0.0037, 0.0017, 0.0125, 0.0026, 0.0117, 0.2161, 0.0096, 0.2504, 0.1103,
        0.3815])

In [86]:
test_label.shape

torch.Size([10000])

In [21]:
tensor = torch.ones(4, 4)
torch.cat([tensor, tensor], dim=1)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [20]:
torch.stack([tensor, tensor], dim=1)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

In [48]:
torch.zeros(10, dtype=torch.int16).scatter_(dim=0, index=torch.tensor(1), src=torch.tensor(2, dtype=torch.int16))

tensor([0, 2, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int16)

In [63]:
src = torch.arange(1, 11).reshape(2, 5)
print(src.shape)
src

torch.Size([2, 5])


tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])

In [62]:
index = torch.tensor([[0, 1, 2, 0]])
print(index.shape)
index

torch.Size([1, 4])


tensor([[0, 1, 2, 0]])

In [64]:
self = torch.zeros(3, 5)
print(self.shape)
self

torch.Size([3, 5])


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [96]:
sz = 3
torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)


tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])